In [ ]:
import pandas as pd
import math
import folium
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from datetime import datetime
import re

m = folium.Map(location=(50.07534086017266, 19.801896715224018), zoom_start=14, control_scale=True)
tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(m)

def odleglosc_euklides(iks_jeden,iks_dwa):
    #returns the distance between two points, 
    # the curvature of the earth is not taken into account 
    # due to small distances (deemed not impactful)
    return math.sqrt((abs(iks_jeden[0]-iks_dwa[0])**2)+(abs(iks_jeden[1]-iks_dwa[1])**2))

def extract_weather(metar):
    #Return the type of weather using a RegExp
    snow_pattern = r'SN'
    clear_pattern = r'CAVOK'
    if re.search(snow_pattern, metar):
        return 'snow'
    elif re.search(clear_pattern, metar):
        return 'cavok'
    else:
        return 'other'

def spawn_polygon(locs,colour):
    return folium.Polygon(locations=locs,
              color=colour,
              weight=0.1,
              fill=True,
              fill_color=colour,
              fill_opacity = 0.2,
              )
def spawn_point(coords,colour_pick):
    return folium.Circle(location=coords,
              radius=1,
              color='black',
              weight=0.3,
              fill=True,
              fill_color=colour_pick,
              fill_opacity = 0.5,
              )

def calculate_move_vector(polygon_cooords,precisions,first_point):
    # this function is used to calculate how long are the sides 
    # of each individual polygon made from cutting up a large polygon (the runway)
    return [
        [0,0],
        [((polygon_cooords[1][0]-first_point[0])/precisions[1]),((polygon_cooords[1][1]-first_point[1])/precisions[1])],
        [(((polygon_cooords[3][0]-polygon_cooords[0][0])/precisions[0])),((polygon_cooords[3][1]-polygon_cooords[0][1])/precisions[0])],
        [(((polygon_cooords[1][0]-polygon_cooords[0][0])/precisions[1])),((polygon_cooords[1][1]-polygon_cooords[0][1])/precisions[1])]]

def find_direction(first_point,second_point,given_direction = -1):
    # this function is used to determine the direction 
    # that the vehicle is facing while traversing the runway
    # based on the current point and the next point
    degrees_of_move = math.atan2( second_point[1] - first_point[1], second_point[0] - first_point[0] ) * ( 180 / math.pi )
    
    #if the input data contains trustworthy directional data, uncomment the two lines below
    #if given_direction >= 0:
    #    degrees_of_move = given_direction
    if degrees_of_move < 0:
        degrees_of_move = 360+degrees_of_move
    if degrees_of_move >= 22.5 and degrees_of_move <= 67.5:
        return "north-east"
    elif degrees_of_move >= 67.5 and degrees_of_move <= 112.5:
        return "east"
    elif degrees_of_move >= 112.5 and degrees_of_move <= 157.5:
        return "south-east"
    elif degrees_of_move >= 157.5 and degrees_of_move <= 202.5:
        return "south"
    elif degrees_of_move >= 202.5 and degrees_of_move <= 247.5:
        return "south-west"
    elif degrees_of_move >= 247.5 and degrees_of_move <= 292.5:
        return "west"
    elif degrees_of_move >= 292.5 and degrees_of_move <= 337.5:
        return "north-west"
    else:
        return "north"

def colour_occurences_by_num(observation_power):
    # this function is purely for aesthetics, 
    # to return a colour from a scale of colours based on 
    # the strength of the observation
    red_string = "FF"
    green_string = "00"
    if observation_power > 0:
        if observation_power > 0.5:
            green_string = "FF"
        else:
            green_string = str(format(math.floor(256*observation_power*2),'x'))
            if len(green_string) < 2:
                green_string = "0" + green_string
    else:
        return '#000000'
    if observation_power > 0.5:
        if observation_power >= 1:
            red_string = "00"
        else:
            red_string = str(format(math.ceil(256-(256*(observation_power-0.5)*2)),'x'))
            if len(red_string) < 2:
                red_string = "0" + red_string
            if len(green_string) < 2:
                green_string = "0" + green_string
    return "#"+red_string+green_string+"00"

def concept_1():
    for i in range(0,len(polygons_hit)):
        if polygons_hit[i] > 1:
            polygons_hit[i] = 1
        elif polygons_hit[i] == 0:
            polygons_hit[i] += 0.0000000001
    print(sum(polygons_hit),len(polygons_hit))
    return sum(polygons_hit)/len(polygons_hit)
def concept_2(direction,only_important = False):
    for i in range(0,len(polygons_hit)):
        if polygons_hit[i] > 1:
             polygons_hit[i] = 1
    sum_of_hits = 0
    num_of_squares = 0
    if direction == "25":
        for i in range(0,int(precision[0][0]/3)):
            for j in range(0,72-(i*5)):
                sum_of_hits += polygons_hit[(i+16)*precision[0][1]+j+710]
                polygons_hit[(i+16)*precision[0][1]+j+715] += 0.0000000000000000001
                sum_of_hits += polygons_hit[(i+16)*precision[0][1]+j+395]
                polygons_hit[(i+16)*precision[0][1]+j+400] += 0.0000000000000000001
                sum_of_hits += polygons_hit[(i+16)*precision[0][1]+j]
                polygons_hit[(i+16)*precision[0][1]+j] += 0.0000000000000000001
                num_of_squares += 3
    elif direction == "07":
        for i in range(0,int(precision[0][0]/3)):
            for j in range(0,72-(i*5)):
                sum_of_hits += polygons_hit[(i+16)*precision[0][1]+(72-j)+1047]
                polygons_hit[(i+16)*precision[0][1]+(72-j)+1047] += 0.0000000000000000001
                sum_of_hits += polygons_hit[(i+16)*precision[0][1]+(72-j)+662]
                polygons_hit[(i+16)*precision[0][1]+(72-j)+662] += 0.0000000000000000001
                sum_of_hits += polygons_hit[(i+16)*precision[0][1]+(72-j)+347]
                polygons_hit[(i+16)*precision[0][1]+(72-j)+347] += 0.0000000000000000001
                num_of_squares += 3
    else:
        return "Error: Unspecified direction"
    for i in range(int(len(polygons_hit)/3),int(len(polygons_hit)/3+(precision[0][1]*precision[0][0]/3))):
        sum_of_hits += polygons_hit[i]
        polygons_hit[i] += 0.0000000000000000001
        num_of_squares += 1
    if only_important == False:
        num_of_squares += len(polygons_hit)
        for i in range(0,len(polygons_hit)):
                if polygons_hit[i] != 0.0000000000000000001:
                    sum_of_hits += polygons_hit[i]
    print(sum_of_hits,num_of_squares)
    return sum_of_hits/num_of_squares

def concept_3(direction,only_important = False):
    for i in range(0,len(polygons_hit)):
        if polygons_hit[i] > 1:
            polygons_hit[i] = 1
    sum_of_hits = 0
    num_of_squares = 0
    if direction == "25":
        for i in range(0,int(precision[0][0]/3)):
            for j in range(0,72-(i*5)):
                #trapez boki
                sum_of_hits += polygons_hit[(i+16)*precision[0][1]+(72-j)+586]
                polygons_hit[(i+16)*precision[0][1]+(72-j)+586] += 0.0000000000000000001
                sum_of_hits += polygons_hit[(int(precision[0][0]/3)-i)*precision[0][1]+(72-j)+586]
                polygons_hit[(int(precision[0][0]/3)-i-1)*precision[0][1]+(72-j)+586] += 0.0000000000000000001
                #zjazd srodek
                sum_of_hits += polygons_hit[(i+16)*precision[0][1]+j+400]
                polygons_hit[(i+16)*precision[0][1]+j+400] += 0.0000000000000000001
                num_of_squares += 3
            for j in range(0,72-(i*3)):
                #trapez boki
                sum_of_hits += polygons_hit[(i+16)*precision[0][1]+j+856]
                polygons_hit[(i+16)*precision[0][1]+j+856] += 0.0000000000000000001        
                sum_of_hits += polygons_hit[(int(precision[0][0]/3)-i)*precision[0][1]+(42+j)+814]
                polygons_hit[(int(precision[0][0]/3)-i-1)*precision[0][1]+(42+j)+814] += 0.0000000000000000001
                num_of_squares += 2
                #srodek
            for j in range(0,140-(i*5)):
                #lejek na koncu
                sum_of_hits += polygons_hit[(int(precision[0][0]/3)-i)*precision[0][1]+j]
                polygons_hit[(int(precision[0][0]/3)-i-1)*precision[0][1]+j] += 0.0000000000000000001
                sum_of_hits += polygons_hit[(i+16)*precision[0][1]+j]
                polygons_hit[(i+16)*precision[0][1]+j] += 0.0000000000000000001
                num_of_squares += 2
                #trapez rownolegle scianki
        for i in range(0,int(precision[0][0]/3)):
            for j in range(0,197):
                sum_of_hits += polygons_hit[i*precision[0][1]+j+659]
                polygons_hit[i*precision[0][1]+j+659] += 0.0000000000000000001
                sum_of_hits += polygons_hit[(i+16)*precision[0][1]+j+659]
                polygons_hit[(i+16)*precision[0][1]+j+659] += 0.0000000001
                num_of_squares += 2
    elif direction == "07":
        for i in range(0,int(precision[0][0]/3)):
            for j in range(0,72-(i*3)):
                sum_of_hits += polygons_hit[(i+16)*precision[0][1]+(72-j)+150]
                polygons_hit[(i+16)*precision[0][1]+(72-j)+150] += 0.0000000000000000001
                sum_of_hits += polygons_hit[(int(precision[0][0]/3)-i)*precision[0][1]+(72-j)+150]
                polygons_hit[(int(precision[0][0]/3)-i-1)*precision[0][1]+(72-j)+150] += 0.0000000000000000001
                num_of_squares += 2
                #zjazdy
            for j in range(0,72-(i*5)):
                sum_of_hits += polygons_hit[(i+16)*precision[0][1]+j+420]
                polygons_hit[(i+16)*precision[0][1]+j+420] += 0.0000000000000000001        
                sum_of_hits += polygons_hit[(int(precision[0][0]/3)-i)*precision[0][1]+(42+j)+378]
                polygons_hit[(int(precision[0][0]/3)-i-1)*precision[0][1]+(42+j)+378] += 0.0000000000000000001
                sum_of_hits += polygons_hit[(i+16)*precision[0][1]+(72-j)+662]
                polygons_hit[(i+16)*precision[0][1]+(72-j)+662] += 0.0000000000000000001
                num_of_squares += 3
            for j in range(0,197):
                sum_of_hits += polygons_hit[i*precision[0][1]+j+223]
                polygons_hit[i*precision[0][1]+j+223] += 0.0000000000000000001
                sum_of_hits += polygons_hit[(i+16)*precision[0][1]+j+223]
                polygons_hit[(i+16)*precision[0][1]+j+223] += 0.0000000001
                num_of_squares += 2
            for j in range(0,140-(i*5)):
                sum_of_hits += polygons_hit[(i+16)*precision[0][1]+(72-j)+1047]
                polygons_hit[(i+16)*precision[0][1]+(72-j)+1047] += 0.0000000000000000001
                sum_of_hits += polygons_hit[(int(precision[0][0]/3)-i)*precision[0][1]+(72-j)+1047]
                polygons_hit[(int(precision[0][0]/3)-i-1)*precision[0][1]+(72-j)+1047] += 0.0000000000000000001
                num_of_squares += 2
    else:
        return "Error: Unspecified direction"
    for i in range(int(len(polygons_hit)/3),int(len(polygons_hit)/3+(precision[0][1]*precision[0][0]/3))):
        sum_of_hits += polygons_hit[i]
        polygons_hit[i] += 0.0000000000000000001
        num_of_squares += 1
    if only_important == False:
        num_of_squares += len(polygons_hit)
        for i in range(0,len(polygons_hit)):
                if polygons_hit[i] != 0.0000000000000000001:
                    sum_of_hits += polygons_hit[i]
    print(sum_of_hits,num_of_squares)
    return sum_of_hits/num_of_squares


def chosen_concept(number,direction,only_important = False):
    for i in range(0,len(polygons_hit)):
        if polygons_hit[i] == 0.0000000000000000001:
            polygons_hit[i] = 0
    if number == 1:
        return round(concept_1()*100,2)
    elif number == 2:
        return round(concept_2(direction,only_important)*100,2)
    else:
        return round(concept_3(direction,only_important)*100,2)

start_czas = "2022-06-11 18:00:00"
koniec_czas = "2022-06-11 18:45:00"
all_data = pd.read_csv('./180_190k.csv',delimiter=";")
metar = pd.read_csv('./metar.csv',delimiter=",")

# into how many squares cut up the entire area of the runway
precision = [[24,1120]]

#the corners of the runway
runway_coords = [[[50.07566710194821, 19.76799713444237],[50.080515119303335, 19.802809237364443],[50.079982483103024, 19.80303356739039],[50.07514837915356, 19.768117444096198]]]

#import data, throw out unnecessary data and discard any datapoints that are too far away to consider
all_data.rename(columns = {'id':'id', 'sensor':'sensor', 'location':'location', 'value1':'szerokosc_geograficzna', 'value2':'temperatura_2', 'value3':'dlugosc_geograficzna', 'value4':'value4', 'value5':'value5', 'value6':'value6', 'value7':'value7', 'value8':'value8', 'value9':'value9', 'czar serwera':'czas_serwer'}, inplace = True) 
all_data = all_data.drop(all_data[all_data.sensor == "ESP32_pojazd2"].index)
all_data = all_data.drop(all_data[all_data.szerokosc_geograficzna < 50].index)
all_data = all_data.drop(all_data[all_data.szerokosc_geograficzna > 51].index)
all_data = all_data.drop(all_data[all_data.dlugosc_geograficzna > 20].index)
all_data = all_data.drop(all_data[all_data.dlugosc_geograficzna < 19.5].index)
temp_data = all_data[["szerokosc_geograficzna","dlugosc_geograficzna","czas_serwer"]]
temp_data['czas_serwer'] = pd.to_datetime(temp_data['czas_serwer'], format="%Y-%m-%d %H:%M:%S")
metar = metar.drop('station', axis=1)
metar['valid'] = pd.to_datetime(metar['valid'], format="%Y-%m-%d %H:%M")
temp_data = temp_data[(temp_data['czas_serwer'] >= pd.to_datetime(start_czas)) & (temp_data['czas_serwer'] <= pd.to_datetime(koniec_czas))]
temp_data = pd.merge_asof(temp_data, metar, left_on='czas_serwer', right_on='valid', direction='nearest')
polygons_coords = []
temp_data['weather'] = temp_data['metar'].apply(extract_weather)
temp_data = temp_data.drop('valid', axis=1)
temp_data = temp_data.drop('metar', axis=1)
temp_data["inspection"] = "no"
temp_data["direction"] = "none"
temp_data["inspection_id"] = "none"
polygons_hit = [0]*len(polygons_coords)
current_inspection_status = "no"
current_direction = "none"
num_of_inspections = 0
num_a = 0 
num_b = len(temp_data.index)-1 
temp_data_midpoints = temp_data
num_of_midpoints = 2

#drawing the individual smaller polygons of the runway
for k in range(0,len(runway_coords)):
    current_point = runway_coords[k][0]
    move_vector = calculate_move_vector(runway_coords[k],precision[k],current_point)
    for i in range(0,precision[k][0]):
        for j in range(0,precision[k][1]):
            a = [current_point[0],current_point[1]]
            b = [current_point[0]+move_vector[1][0],current_point[1]+move_vector[1][1]]
            d = [current_point[0]+move_vector[2][0],current_point[1]+move_vector[2][1]]
            c = [d[0]+move_vector[3][0],d[1]+move_vector[3][1]]
            polygons_coords.append([a[:],b[:],c[:],d[:]])
            current_point = b[:]
            a.clear()
            b.clear()
            c.clear()
            d.clear()
        current_point = [runway_coords[k][0][0]+(move_vector[2][0]*(i+1)),runway_coords[k][0][1]+(move_vector[2][1]*(i+1))]
polygon_length = calculate_move_vector(runway_coords[0],precision[0],runway_coords[0][0])

colors = ["#ff0000","#00ff00","#0000ff"]

#draw a line between datapoints
punkty_linii = []
for i in range(num_a,num_b):
    punkty_linii.append((temp_data.iloc[i]["szerokosc_geograficzna"],temp_data.iloc[i]["dlugosc_geograficzna"]))
folium.PolyLine(locations=punkty_linii,weight=1,color="#000000").add_to(m)


#setting some basic variables 
polygons_hit = [0]*len(polygons_coords)
range_width = 4
range_jump = 1
range_min = 1
base_observation_strength = 0.6
visibility_parameter = 1
time_current = temp_data_midpoints.iloc[num_b]["czas_serwer"]
total_observation_strength = base_observation_strength*visibility_parameter
optimise_check_length = 10
optimisation_iterator = -1
starting_point = 0
optimise_vector = calculate_move_vector(runway_coords[0],precision[0],runway_coords[0][0])
found_match = False
current_cords = []



#the main calculating part
for i in range(num_a,num_b):
    current_cords = [temp_data_midpoints.iloc[i]["szerokosc_geograficzna"],temp_data_midpoints.iloc[i]["dlugosc_geograficzna"]]
    difference = [(temp_data_midpoints.iloc[i]["szerokosc_geograficzna"]-temp_data_midpoints.iloc[i+1]["szerokosc_geograficzna"])/(num_of_midpoints+1),(temp_data_midpoints.iloc[i]["dlugosc_geograficzna"]-temp_data_midpoints.iloc[i+1]["dlugosc_geograficzna"])/(num_of_midpoints+1)]
    
    #If the vehicle is out of bounds, then skip this datapoint, no need for calculations
    if temp_data_midpoints.iloc[i]["szerokosc_geograficzna"] < 50.07514837915356 or temp_data_midpoints.iloc[i]["szerokosc_geograficzna"] < 50.07828583169408 and temp_data_midpoints.iloc[i]["dlugosc_geograficzna"] > 19.793968884507432:
        continue
    #Creating midpoints with interpolation
    for j in range(0,num_of_midpoints+1):
        found_match = False
        x_sr = math.ceil((current_cords[0]-runway_coords[0][0][0])/polygon_length[1][0])
        starting_point = precision[0][1]-round((runway_coords[0][2][1]-current_cords[1])/optimise_vector[1][1])-round(optimise_check_length/2)
        k = starting_point
        optimisation_iterator -= 1
        spawn_point([current_cords[0],current_cords[1]],"#00ff00").add_to(m)
        #Finding which square the datapoints belongs to
        while k <len(polygons_coords):
            if Polygon(polygons_coords[k]).contains(Point(current_cords[0],current_cords[1])):
                polygons_hit[k] += total_observation_strength
                found_match = True
                direction_facing = find_direction([current_cords[0],current_cords[1]],[temp_data_midpoints.iloc[i+1]["szerokosc_geograficzna"],temp_data_midpoints.iloc[i+1]["dlugosc_geograficzna"]])
                #Adding the observation strength to the squares, to make them count as seen
                if direction_facing == "west":
                    for w in range(1,range_width):
                        for h in range(-w,w+1):
                            target_square = k-(precision[0][1]*h)-w
                            if target_square < len(polygons_hit) and target_square >= 0:
                                polygons_hit[target_square] += total_observation_strength
                    for w in range(1,range_width+2):
                        for h in range(-range_width+w-1,range_width-w):
                            target_square = k+(precision[0][1]*(h+1))-w-range_width+1
                            if target_square < len(polygons_hit) and target_square >= 0:
                                polygons_hit[target_square] += total_observation_strength
                    for w in range(1,range_width):
                        for h in range(-round(range_width/2)+1,round(range_width/2)): 
                            target_square = k-(precision[0][1]*h)-w
                            if target_square < len(polygons_hit) and target_square >= 0:
                                polygons_hit[target_square] += total_observation_strength
                    for w in range(1,range_width*2):
                        for h in range(-round(range_width/2)+1,round(range_width/2)): 
                            target_square = k-(precision[0][1]*h)-w
                            if target_square < len(polygons_hit) and target_square >= 0:
                                polygons_hit[target_square] += total_observation_strength
                elif direction_facing == "north":
                    for h in range(1,range_width):
                        for w in range(-h,h):
                            target_square = k-(precision[0][1]*h)+w
                            if target_square < len(polygons_hit) and target_square >= 0:
                                polygons_hit[target_square] += total_observation_strength  
                    for h in range(1,range_width*2):
                        for w in range(-round(range_width/2),round(range_width/2)):
                            target_square = k-(precision[0][1]*h)+w
                            if target_square < len(polygons_hit) and target_square >= 0:
                                polygons_hit[target_square] += total_observation_strength
                elif direction_facing == "north-west":
                    for w in range(1,range_width):
                        for h in range(0,w*2):
                            target_square = k-(precision[0][1]*h)-w
                            if target_square < len(polygons_hit) and target_square >= 0:
                                polygons_hit[target_square] += total_observation_strength  
                    for w in range(1,range_width+1):
                        for h in range(w-1,w*2+1):
                            target_square = k-(precision[0][1]*h)-w
                            if target_square < len(polygons_hit) and target_square >= 0:
                                polygons_hit[target_square] += total_observation_strength
                elif direction_facing == "north-east":
                    for w in range(1,range_width):
                        for h in range(0,w*2):
                            target_square = k-(precision[0][1]*h)+w
                            if target_square < len(polygons_hit) and target_square >= 0:
                                polygons_hit[target_square] += total_observation_strength  
                    for w in range(1,range_width+1):
                        for h in range(w-1,w*2+1):
                            target_square = k-(precision[0][1]*h)+w
                            if target_square < len(polygons_hit) and target_square >= 0:
                                polygons_hit[target_square] += total_observation_strength  
                elif direction_facing == "south":
                    for h in range(1,range_width):
                        for w in range(-h,h+1):
                            target_square = k+(precision[0][1]*h)+w
                            if target_square < len(polygons_hit) and target_square >= 0:
                                polygons_hit[target_square] += total_observation_strength
                    for h in range(1,range_width+1):
                        for w in range(-range_width+h,range_width-h+1):
                            target_square = k+(precision[0][1]*(h+range_width-1))+w
                            if target_square < len(polygons_hit) and target_square >= 0:
                                polygons_hit[target_square] += total_observation_strength
                    for h in range(1,range_width*2):
                        for w in range(-round(range_width/2)+1,round(range_width/2)):
                            target_square = k+(precision[0][1]*h)+w
                            if target_square < len(polygons_hit) and target_square >= 0:
                                polygons_hit[target_square] += total_observation_strength
                    for h in range(1,range_width):
                        for w in range(-round(range_width/2)+1,round(range_width/2)):
                            target_square = k+(precision[0][1]*h)+w
                            if target_square < len(polygons_hit) and target_square >= 0:
                                polygons_hit[target_square] += total_observation_strength
                elif direction_facing == "south-west":
                    for w in range(1,range_width):
                        for h in range(0,w*2):
                            target_square = k+(precision[0][1]*h)-w
                            if target_square < len(polygons_hit) and target_square >= 0:
                                polygons_hit[target_square] += total_observation_strength
                    for w in range(0,range_width+1):
                        for h in range(w-1,w*2+1):
                            target_square = k+(precision[0][1]*h)-w
                            if target_square < len(polygons_hit) and target_square >= 0:
                                polygons_hit[target_square] += total_observation_strength
                elif direction_facing == "south-east":
                    for w in range(1,range_width):
                        for h in range(0,w*2):
                            target_square = k+(precision[0][1]*h)+w
                            if target_square < len(polygons_hit) and target_square >= 0:
                                polygons_hit[target_square] += total_observation_strength
                    for w in range(0,range_width+1):
                        for h in range(w-1,w*2+1):
                            target_square = k+(precision[0][1]*h)+w
                            if target_square < len(polygons_hit) and target_square >= 0:
                                polygons_hit[target_square] += total_observation_strength
                elif direction_facing == "east":
                    for w in range(1,range_width):
                        for h in range(-w,w+1):
                            target_square = k+(precision[0][1]*h)+w
                            if target_square < len(polygons_hit) and target_square >= 0:
                                polygons_hit[target_square] += total_observation_strength
                    for w in range(1,range_width+2):
                        for h in range(-range_width+w-1,range_width-w):
                            target_square = k+(precision[0][1]*(h+1))+w+range_width-1
                            if target_square < len(polygons_hit) and target_square >= 0:
                                polygons_hit[target_square] += total_observation_strength
                    for w in range(1,range_width):
                        for h in range(-round(range_width/2)+1,round(range_width/2)): 
                            target_square = k+(precision[0][1]*h)+w
                            if target_square < len(polygons_hit) and target_square >= 0:
                                polygons_hit[target_square] += total_observation_strength
                    for w in range(1,range_width*2):
                        for h in range(-round(range_width/2)+1,round(range_width/2)): 
                            target_square = k+(precision[0][1]*h)+w
                            if target_square < len(polygons_hit) and target_square >= 0:
                                polygons_hit[target_square] += total_observation_strength
                else:
                    print("An error has occurred.")
                optimisation_iterator = -1
                break
            optimisation_iterator += 1
            if optimisation_iterator >= optimise_check_length:
                k += precision[0][1] - optimise_check_length
                optimisation_iterator = -1
            else:
                k+=1
        current_cords[0] -= difference[0]
        current_cords[1] -= difference[1]

#generating the results

wynik = [chosen_concept(1,"25",False),chosen_concept(2,"25",False),chosen_concept(2,"25",True),chosen_concept(3,"25",False),chosen_concept(3,"25",True)]
for i in range(0,len(polygons_coords)):
            spawn_polygon(polygons_coords[i],colour_occurences_by_num(polygons_hit[i])).add_to(m)
legend_html = f'''
     <div style="position: fixed;
                 top: 50px; left: 50px;
                 background-color: rgba(255, 255, 255, 0.8);
                 border-radius: 5px;
                 z-index: 9999;
                 font-size: 14px;
                 padding: 10px;
                 ">
     Dane od:<br>
     {start_czas}<br>
     do:<br>
     {koniec_czas}<br>
     W1:{wynik[0]}%<br>
     W2:{wynik[1]}%<br>
     W3:{wynik[2]}%<br>
     W4:{wynik[3]}%<br>
     W5:{wynik[4]}%<br>
     </div>
'''

m.get_root().html.add_child(folium.Element(legend_html))
m.save("plik.html")

display(m)




